In [1]:
import pandas as pd
from tqdm import tqdm
from neo4j import GraphDatabase
import json
import re
import os
import glob

In [4]:

path = "./"
all_files = glob.glob(os.path.join(path, "./Datasets/api_comic_list*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "./Datasets/merged.csv")

In [2]:
db_host = "bolt://localhost:7687"
password = "1234qwer"


In [7]:
api_characters = pd.read_csv("./Datasets/api_character_list.csv")
scraping_characters = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_relations.csv")
api_comics = pd.read_csv("./Datasets/merged.csv")


In [ ]:
scraping_characters.head(20)

In [ ]:
api_characters.head(20)

Add characters nodes with variant from API

In [8]:
def add_character_node(tx, name, character_id, character_description):
  processed_name = name.replace('"', " ").replace('"'," ")
  nome_senza_parentesi = re.sub("[\(\[].*?[\)\]]", "", processed_name.strip())
  query_string = 'MERGE (c:character{name: "%s"})' %(nome_senza_parentesi.replace("'", " ").lower().strip())
  tx.run(query_string)
  tx.run(
      "MERGE (n:character_variant {name: '%s', character_id: %s, character_description: '%s'})" %(name.replace("'", ' ').replace('"', " ").replace("-", " "), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ")))
  query_variant = 'MATCH (a:character{name:"%s"}),(b:character_variant {name: "%s", character_id: %s, character_description: "%s"}) MERGE (b)-[r:variante_di]->(a)' % (
      nome_senza_parentesi.replace("'", " ").lower().strip(), name.replace("'", ' ').replace('"', " ").replace("-", " ").strip(), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ").strip(),)
  tx.run(query_variant)


In [9]:
def add_comic_node_and_link(tx, comic_title, comic_id, character_id, comic_description):
  comic_description = str(comic_description).replace("'", " ").replace('"',"").replace("-"," ").strip()
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s, comic_description: "%s"})' % (
      comic_title, comic_id, comic_description)
  create_link_query = "MATCH (a:character_variant),(b:comic) WHERE a.character_id = %s AND b.comic_id = %s MERGE (a)-[r:Presente]->(b)" % (
      character_id, comic_id)
  # print(create_link_query)
  node_creation_result = tx.run(create_comic_node_query)
  link_creation_result = tx.run(create_link_query)


In [10]:
def add_comic_node(tx, comic_title,comic_id, comic_description):
  comic_description = str(comic_description).replace(
      "'", " ").replace('"', "").replace("-", " ").strip()
  create_comic_node_query = 'MERGE (c:comic {title:"%s", comic_id: %s, comic_description: "%s"})' % (
      comic_title, comic_id, comic_description)
  node_creation_result = tx.run(create_comic_node_query)


In [20]:
def character_present(tx, character):
  res = tx.run("Match (n:character) where n.name='%s' return n"%(character))
  if res.single():
    return True
  else:
    return False


In [11]:
driver = GraphDatabase.driver(db_host, auth=("neo4j",password))

In [65]:

for idx, row in tqdm(api_characters.iterrows(), total=api_characters.shape[0]):
    with driver.session() as session:
        try:
            session.write_transaction(
                add_character_node, row['name'], row["id"], row["description"])
        except Exception as e:
            print(e
            )

100%|██████████| 1559/1559 [01:02<00:00, 24.76it/s]


In [ ]:
for idx, row in tqdm(api_comics.iterrows(), total=api_comics.shape[0]):
    string = row["characters"]
    
    characters = json.loads(string)
    if characters["items"]:
        for character in characters["items"]:
            with driver.session() as session:
                try:
                    character_id = character["resourceURI"].split("/")
                    character_id = character_id[len(character_id) -1]
                    session.write_transaction(
                            add_comic_node_and_link, row['title'], row["id"], character_id,  row["description"])
                except Exception as e:
                    print(e)
    else:
        with driver.session() as session:
            try:
                session.write_transaction(
                    add_comic_node, row['title'], row["id"],  row["description"])
            except Exception as e:
                print(e)
    


In [19]:
def add_character_node(tx, name ):
    create_comic_node_query = 'MERGE (c:character {name:"%s", wiki: true })' %(name)
    tx.run(create_comic_node_query)

def add_variant_node(tx, name, variant_name):
    tx.run('MERGE (b:character_variant {name: "%s", wiki: true})'%(variant_name))
    query_variant = 'MATCH (a:character{name:"%s"}),(b:character_variant {name: "%s", wiki: true}) MERGE (b)-[r:variante_di]->(a)' % (
       name, variant_name)
    tx.run(query_variant)


In [21]:
driver = GraphDatabase.driver(db_host, auth=("neo4j",password))
num_present = 0
with driver.session() as session:
    try:
        character_data = pd.read_csv(
            "./Datasets/scraping_personaggi_completo_filtered_relations.csv")
        for idx, row in tqdm(character_data.iterrows(), total=character_data.shape[0]):
            name = row["Nome"]
            name = name.replace('"', " ").replace("'", " ")
            nome_senza_parentesi = re.sub("[\(\[].*?[\)\]]", "", name.strip())
            nome_senza_parentesi = nome_senza_parentesi.strip().lower()
            present = session.write_transaction(
                character_present, nome_senza_parentesi)
            if present:
                num_present +=1
                session.write_transaction(add_variant_node, nome_senza_parentesi, name)
            else:
                session.write_transaction(add_character_node, nome_senza_parentesi)
    except Exception as e:
        print(e
        )
print(num_present)

100%|██████████| 3746/3746 [04:40<00:00, 13.37it/s]

3746


Insert data into database, link movie found into character csv movie_list to movie node, character in database must have wiki true.

In [21]:
#charachter_name must have " and ' replaced with whytespace and should be trimmed
#movie_title must have " and ' replaced with whytespace and should be trimmed
def add_movie_node_and_link_character(tx, movie_title, character_name):
  #create relation between movie anc character if mot exists
  tx.run("MATCH (m:movie), (c:character_variant) WHERE m.title = '%s' AND c.name = '%s' MERGE (c)-[r:in_film]->(m)"%(movie_title, character_name))

#movie_title must have " and ' and - replaced with whytespace and should be trimmed
def add_movie(tx, movie_title, release_date, runtime, box_office, synopsis):
  #create movie node if not exists
  tx.run("MERGE (n:movie {title: '%s', release_date: '%s', screen_time: '%s', box_office: '%s', synopsis: '%s' })" % (movie_title, release_date, runtime, box_office, synopsis))
  

In [19]:
import numpy as np
driver = GraphDatabase.driver(db_host, auth=("neo4j", password))
num_present = 0
with driver.session() as session:
    try:
        film_data = pd.read_csv(
            "./Datasets/film_data_scraping.csv")
        for idx, row in tqdm(film_data.iterrows(), total=film_data.shape[0]):
          
            film_title = row["Title"].strip() if str(row["Title"]) != "nan" else None
            film_release_date = row["releaseDate"].strip(
            ) if str(row["releaseDate"]) != "nan" else None
            film_runtime = row["Runtime"].strip() if str(
                row["Runtime"]) != "nan" else None
            film_box_office = row["BoxOffice"].strip(
            ) if str(row["BoxOffice"]) != "nan" else None
            film_synopsis = json.loads(row["Synopsis"])[
                0] if str(row["Synopsis"]) != "nan" else None
            #remove illegal string characters
            if film_title:
                film_title = film_title.replace('"', " ").replace("'", " ").replace("-"," ").strip()
            #remove useless characters from synopsis
            if film_synopsis:
                film_synopsis = film_synopsis.replace("-","\-").replace("'","\\'").replace('"','\\"').strip()
            session.write_transaction(
                    add_movie, film_title, film_release_date, film_runtime, film_box_office, film_synopsis)
            
    except Exception as e:
        print(e)


100%|██████████| 42/42 [00:04<00:00, 10.24it/s]

0


Link character with movies

In [22]:
driver = GraphDatabase.driver(db_host, auth=("neo4j", password))
num_present = 0
with driver.session() as session:
    try:
        character_data = pd.read_csv(
            "./Datasets/scraping_personaggi_completo_filtered_relations.csv")
        for idx, row in tqdm(character_data.iterrows(), total=character_data.shape[0]):
            films = json.loads(row["Film"])
            name = row["Nome"]
            name = name.replace('"', " ").replace("'", " ")
            for film in films:
                film_name = film[0]
                film_name = film_name.replace('"', " ").replace(
                    "'", " ").replace("-", " ").strip()
                session.write_transaction(add_movie_node_and_link_character, film_name, name)           
    except Exception as e:
        print(e
              )


100%|██████████| 3746/3746 [00:56<00:00, 66.25it/s] 


# Aggiunta delle serie nel database


Creazione per ogni serie di un nodo di tipo *serie* contenente le varie info ottenute e collegamento ai nodi *character variant* corrispondenti

---

### Definizione delle funzioni
Definizione delle funzioni utilizzate per l'inserimento nel database.

In [32]:
import re
def escape_for_neo4j(string):
  return string.replace('"', '\\"').replace("'", "\\'").replace("-", "\\-").strip()
def create_node_tv_serie(tx, title, creators, showrunners, producers, composers, release_date):
  creators_list = list(map(lambda item: re.escape(item[0]).replace("\ "," ").strip(), creators))
  showrunners_list = list(
      map(lambda item: escape_for_neo4j(item[0]).strip(), showrunners))
  producers_list = list(
      map(lambda item: escape_for_neo4j(item[0]).strip(), producers))
  composers_list = list(
      map(lambda item: escape_for_neo4j(item[0]).strip(), composers))
  #remove illegal characters 
  title = escape_for_neo4j(title)
  release_date = escape_for_neo4j(release_date)
  #prepare query
  query_string = "MERGE (n:tv_show {title:'%s', creators:%s, showrunners: %s, producers: %s, composers: %s, release_date: '%s'})"%(title, creators_list, showrunners_list, producers_list,composers_list, release_date)
  tx.run(query_string)
#name = name.replace('"', " ").replace("'", " ")
def link_character_and_series(tx, character_name, serie_name):
  character_name = character_name.replace('"', " ").replace("'", " ")
  serie_name = escape_for_neo4j(serie_name)
  query_string = "MATCH (c:character_variant), (s:tv_show) WHERE c.name = '%s' AND s.title = '%s' MERGE (c)-[r:in_serie]->(s)"%(character_name, serie_name)
  tx.run(query_string)


### Add series nodes

In [33]:
tv_series_dataframe = pd.read_csv("./Datasets/tv_series.csv")

driver = GraphDatabase.driver(db_host, auth=("neo4j", password))
with driver.session() as session:
  try:
    for idx, row in tqdm(tv_series_dataframe.iterrows(), total=tv_series_dataframe.shape[0]):
      try:
        title = row["Title"]
        creators = json.loads(row["Creators"])
        Showrunners = json.loads(row["Showrunners"])
        Producers = json.loads(row["Producers"])
        Composers = json.loads(row["Composers"])
        release_date = row["releaseDate"]
        session.write_transaction(create_node_tv_serie,title, creators, Showrunners, Producers, Composers, str(release_date))
      except Exception as e:
        print(e)
  except Exception as e:
    print(e)


100%|██████████| 19/19 [00:02<00:00,  6.79it/s]


### link characters with series

In [35]:
characters_dataframe = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_relations.csv")

driver = GraphDatabase.driver(db_host, auth=("neo4j", password))
error_log = []
with driver.session() as session:
  try:
    for idx, row in tqdm(characters_dataframe.iterrows(), total=characters_dataframe.shape[0]):
      try:
        series = json.loads(row["Serie"])
        character_name = row["Nome"]
        series = list(map(lambda item: escape_for_neo4j(item[0]), series))
        for item in series:
          session.write_transaction(link_character_and_series, character_name, item)
      except Exception as e:
        error_log.append(e)
  except Exception as e:
    print(e)
print(error_log)


100%|██████████| 3746/3746 [03:13<00:00, 19.34it/s]

[CypherSyntaxError('Invalid input \' \': expected \'m/M\' or \'t/T\' (line 1, column 103 (offset: 102))\r\n"MATCH (c:character_variant), (s:tv_show) WHERE c.name = \'Adrian (Trainer)\' AND s.title = \'AKA You\\\\\'re Welcome\' MERGE (c)-[r:in_serie]->(s)"\r\n                                                                                                       ^'), CypherSyntaxError('Invalid input \'t\': expected whitespace, \'.\', node labels or rel types, \'[\', \'^\', \'*\', \'/\', \'%\', \'+\', \'-\', "=~", IN, STARTS, ENDS, CONTAINS, IS, \'=\', "<>", "!=", \'<\', \'>\', "<=", ">=", AND, XOR, OR, USE GRAPH, LOAD CSV, START, MATCH, UNWIND, MERGE, CREATE UNIQUE, CREATE, SET, DELETE, REMOVE, FOREACH, WITH, CALL, RETURN, UNION, \';\' or end of input (line 1, column 92 (offset: 91))\r\n"MATCH (c:character_variant), (s:tv_show) WHERE c.name = \'Aisha Axton\' AND s.title = \'Can\\\\\'t Front on Me\' MERGE (c)-[r:in_serie]->(s)"\r\n                                                           